In [2]:
import gc
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import matthews_corrcoef
from operator import itemgetter


/Users/hkwik/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
%connect_info

{
  "stdin_port": 52532, 
  "ip": "127.0.0.1", 
  "control_port": 52533, 
  "hb_port": 52534, 
  "signature_scheme": "hmac-sha256", 
  "key": "6b4f8d4a-d449-41a7-b8a9-a5770fb5e975", 
  "kernel_name": "", 
  "shell_port": 52530, 
  "transport": "tcp", 
  "iopub_port": 52531
}

Paste the above JSON into a file, and connect with:
    $> ipython <app> --existing <file>
or, if you are local, you can connect with just:
    $> ipython <app> --existing /Users/hkwik/Library/Jupyter/runtime/kernel-b145caa8-4b3c-4a0d-85db-bf01a97a3760.json 
or even just:
    $> ipython <app> --existing 
if this is the most recent IPython session you have started.


In [6]:
def get_date_features():
    directory = 'data/'
    trainfile = 'train_date.csv'
    
    for i, chunk in enumerate(pd.read_csv(directory + trainfile,
                                          chunksize=1,
                                          low_memory=False)):
        features = list(chunk.columns)
        break

    seen = np.zeros(52)
    rv = []
    for f in features:
        if f == 'Id' or 'S24' in f or 'S25' in f:
            rv.append(f)
            continue
            
        station = int(f.split('_')[1][1:])
#        print(station)
        
        if seen[station]:
            continue
        
        seen[station] = 1
        rv.append(f)
        
    return rv
        
usefuldatefeatures = get_date_features()

In [20]:
tmp = pd.read_csv('data/train_numeric.csv', usecols=['Id', 'Response'])
#test_tmp = pd.read_csv('data/test_numeric.csv', usecols=['Id'])
#test_tmp['Respon']
tmp

,Id,Response
0,4,0
1,6,0
2,7,0
3,9,0
4,11,0
5,13,0
6,14,0
7,16,0
8,18,0
9,23,0


In [23]:
merged = pd.merge(tmp, df_mindate, on='Id')
merged.loc[merged.Response == 0]

,Id,Response,mindate,mindate_id_diff,mindate_id_diff_reverse
0,4,0,82.24,-2314450.0,-224451.0
1,6,0,1313.12,-2284042.0,-67530.0
2,7,0,1618.70,-2313512.0,-26876.0
3,9,0,1149.20,-2196581.0,-3564.0
4,11,0,602.64,-2226678.0,-1.0
5,13,0,1331.66,-2307035.0,-9079.0
6,14,0,1662.63,-2363398.0,-6285.0
7,16,0,791.22,-2094956.0,-101101.0
8,18,0,517.64,-2356186.0,-121525.0
9,23,0,156.27,-2363501.0,-3558.0


In [22]:
merged.loc[merged.Response == 1]

,Id,Response,mindate,mindate_id_diff,mindate_id_diff_reverse
539,1053,1,298.91,-2176553.0,-8355.0
633,1250,1,619.18,1.0,-1.0
681,1350,1,640.20,-2304535.0,-1.0
887,1793,1,1153.46,-2355033.0,-1.0
1152,2347,1,961.65,1.0,-1.0
1731,3527,1,251.12,-2320705.0,-1.0
1759,3585,1,327.08,-62002.0,-1.0
2028,4104,1,200.62,-2317757.0,-1.0
2469,4978,1,680.65,1.0,-1.0
2479,4998,1,1600.39,2544.0,-10160.0


In [25]:
df_mindate.head()

,Id,mindate,mindate_id_diff,mindate_id_diff_reverse
255472,510783,0.00,NaN,-140759.0
325556,651542,0.00,140759.0,543349.0
54038,108193,0.01,-543349.0,-322219.0
215108,430412,0.01,322219.0,-13085.0
221628,443497,0.01,13085.0,-69868.0


In [59]:
base = tmp.copy()

In [72]:
base_test = pd.read_csv('data/test_categorical.csv', nrows=10000, index_col=0, low_memory=False)

In [76]:
base_test.loc[6171].unique()


array([nan, 'T1'], dtype=object)

In [78]:
base[base.Id == 7944]

,Id,Response,cat0,cat1,cat2,cat3,cat4
3969,7944,1,T1,NaN,NaN,NaN,NaN


In [79]:
base[base.Id == 7945]

,Id,Response,cat0,cat1,cat2,cat3,cat4
3970,7945,0,T1,NaN,NaN,NaN,NaN


In [60]:
# load categoricals by chunk
def notnull_colsbyrow(df, index):
    present = df.loc[index].notnull()
    colindices = present[present == True].index
    return df.loc[index,colindices]

cat_iter = pd.read_csv('data/train_categorical.csv', chunksize=10000, index_col=0)


for chunk in cat_iter:
    for index in chunk.index:
        present = notnull_colsbyrow(chunk, index)

        count = 0
        for i in present.unique():
            base.loc[index, 'cat%d' % count] = i
            count += 1
            
        #base.loc[index, 'cat_uniques'] = list(present.unique())
        #if index > 1100:
        #    break
    break

In [64]:
cat_merge = pd.merge(base, df_mindate, on='Id')
cat_merge[cat_merge.Response == 0]

,Id,Response,cat0,cat1,cat2,cat3,cat4,mindate,mindate_id_diff,mindate_id_diff_reverse
0,4,0,NaN,NaN,NaN,NaN,NaN,82.24,-2314450.0,-224451.0
1,6,0,NaN,NaN,NaN,NaN,NaN,1313.12,-2284042.0,-67530.0
2,7,0,NaN,NaN,NaN,NaN,NaN,1618.70,-2313512.0,-26876.0
3,9,0,NaN,NaN,NaN,NaN,NaN,1149.20,-2196581.0,-3564.0
4,11,0,NaN,NaN,NaN,NaN,NaN,602.64,-2226678.0,-1.0
5,13,0,NaN,NaN,NaN,NaN,NaN,1331.66,-2307035.0,-9079.0
6,14,0,NaN,NaN,NaN,NaN,NaN,1662.63,-2363398.0,-6285.0
7,16,0,T1,NaN,NaN,NaN,NaN,791.22,-2094956.0,-101101.0
8,18,0,NaN,NaN,NaN,NaN,NaN,517.64,-2356186.0,-121525.0
9,23,0,T1,NaN,NaN,NaN,NaN,156.27,-2363501.0,-3558.0


In [65]:
cat_merge[cat_merge.Response == 1]

,Id,Response,cat0,cat1,cat2,cat3,cat4,mindate,mindate_id_diff,mindate_id_diff_reverse
539,1053,1,NaN,NaN,NaN,NaN,NaN,298.91,-2176553.0,-8355.0
633,1250,1,T1,NaN,NaN,NaN,NaN,619.18,1.0,-1.0
681,1350,1,T1,NaN,NaN,NaN,NaN,640.20,-2304535.0,-1.0
887,1793,1,T1,NaN,NaN,NaN,NaN,1153.46,-2355033.0,-1.0
1152,2347,1,NaN,NaN,NaN,NaN,NaN,961.65,1.0,-1.0
1731,3527,1,NaN,NaN,NaN,NaN,NaN,251.12,-2320705.0,-1.0
1759,3585,1,NaN,NaN,NaN,NaN,NaN,327.08,-62002.0,-1.0
2028,4104,1,NaN,NaN,NaN,NaN,NaN,200.62,-2317757.0,-1.0
2469,4978,1,NaN,NaN,NaN,NaN,NaN,680.65,1.0,-1.0
2479,4998,1,NaN,NaN,NaN,NaN,NaN,1600.39,2544.0,-10160.0


In [7]:
def get_mindate():
    directory = 'data/'
    trainfile = 'train_date.csv'
    testfile = 'test_date.csv'
    
    features = None
    subset = None
    
    for i, chunk in enumerate(pd.read_csv(directory + trainfile,
                                          usecols=usefuldatefeatures,
                                          chunksize=50000,
                                          low_memory=False)):
        print(i)
        
        if features is None:
            features = list(chunk.columns)
            features.remove('Id')
        
        df_mindate_chunk = chunk[['Id']].copy()
        df_mindate_chunk['mindate'] = chunk[features].min(axis=1).values
        
        if subset is None:
            subset = df_mindate_chunk.copy()
        else:
            subset = pd.concat([subset, df_mindate_chunk])
            
        del chunk
        gc.collect()

    for i, chunk in enumerate(pd.read_csv(directory + testfile,
                                          usecols=usefuldatefeatures,
                                          chunksize=50000,
                                          low_memory=False)):
        print(i)
        
        df_mindate_chunk = chunk[['Id']].copy()
        df_mindate_chunk['mindate'] = chunk[features].min(axis=1).values
        subset = pd.concat([subset, df_mindate_chunk])
        
        del chunk
        gc.collect()      
        
    return subset


df_mindate = get_mindate()

# Group by mindate and id
df_mindate.sort_values(by=['mindate', 'Id'], inplace=True)

# Subtracts id from previous id
df_mindate['mindate_id_diff'] = df_mindate.Id.diff()

# Make a NaN array with the same shape as df_mindate.mindate_id_diff
# Fill it with the sign flipped, starting at index 1
midr = np.full_like(df_mindate.mindate_id_diff.values, np.nan)
midr[0:-1] = -df_mindate.mindate_id_diff.values[1:]

df_mindate['mindate_id_diff_reverse'] = midr

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [1]:
def mcc(tp, tn, fp, fn):
    sup = tp * tn - fp * fn
    inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if inf == 0:
        return 0
    else:
        return sup / np.sqrt(inf)


def eval_mcc(y_true, y_prob, show=False):
    idx = np.argsort(y_prob)
    y_true_sort = y_true[idx]
    n = y_true.shape[0]
    nump = 1.0 * np.sum(y_true)  # number of positive
    numn = n - nump  # number of negative
    tp = nump
    tn = 0.0
    fp = numn
    fn = 0.0
    best_mcc = 0.0
    best_id = -1
    mccs = np.zeros(n)
    for i in range(n):
        if y_true_sort[i] == 1:
            tp -= 1.0
            fn += 1.0
        else:
            fp -= 1.0
            tn += 1.0
        new_mcc = mcc(tp, tn, fp, fn)
        mccs[i] = new_mcc
        if new_mcc >= best_mcc:
            best_mcc = new_mcc
            best_id = i
    if show:
        best_proba = y_prob[idx[best_id]]
        y_pred = (y_prob > best_proba).astype(int)
        return best_proba, best_mcc, y_pred
    else:
        return best_mcc


def mcc_eval(y_prob, dtrain):
    y_true = dtrain.get_label()
    best_mcc = eval_mcc(y_true, y_prob)
    return 'MCC', best_mcc


def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()


def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance


def LeaveOneOut(data1, data2, columnName, useLOO=False):
    grpOutcomes = data1.groupby(columnName)['Response'].mean().reset_index()
    grpCount = data1.groupby(columnName)['Response'].count().reset_index()
    grpOutcomes['cnt'] = grpCount.Response
    if(useLOO):
        grpOutcomes = grpOutcomes[grpOutcomes.cnt > 1]
    grpOutcomes.drop('cnt', inplace=True, axis=1)
    outcomes = data2['Response'].values
    x = pd.merge(data2[[columnName, 'Response']], grpOutcomes,
                 suffixes=('x_', ''),
                 how='left',
                 on=columnName,
                 left_index=True)['Response']
    if(useLOO):
        x = ((x*x.shape[0])-outcomes)/(x.shape[0]-1)
        #  x = x + np.random.normal(0, .01, x.shape[0])
    return x.fillna(x.mean())


def GrabData():
    directory = 'data/'
    trainfiles = ['train_categorical.csv',
                  'train_date.csv',
                  'train_numeric.csv']
    testfiles = ['test_categorical.csv',
                 'test_date.csv',
                 'test_numeric.csv']

    cols = [['Id',
             'L1_S24_F1559', 'L3_S32_F3851',
             'L1_S24_F1827', 'L1_S24_F1582',
             'L3_S32_F3854', 'L1_S24_F1510',
             'L1_S24_F1525'],
            ['Id',
             'L3_S30_D3496', 'L3_S30_D3506',
             'L3_S30_D3501', 'L3_S30_D3516',
             'L3_S30_D3511'],
            ['Id',
             'L1_S24_F1846', 'L3_S32_F3850',
             'L1_S24_F1695', 'L1_S24_F1632',
             'L3_S33_F3855', 'L1_S24_F1604',
             'L3_S29_F3407', 'L3_S33_F3865',
             'L3_S38_F3952', 'L1_S24_F1723',
             'Response']]
    traindata = None
    testdata = None
    for i, f in enumerate(trainfiles):
        print(f)
        subset = None
        for i, chunk in enumerate(pd.read_csv(directory + f,
                                              usecols=cols[i],
                                              chunksize=50000,
                                              low_memory=False)):
            print(i)
            if subset is None:
                subset = chunk.copy()
            else:
                subset = pd.concat([subset, chunk])
            del chunk
            gc.collect()
        if traindata is None:
            traindata = subset.copy()
        else:
            traindata = pd.merge(traindata, subset.copy(), on="Id")
        del subset
        gc.collect()
    del cols[2][-1]  # Test doesn't have response!
    for i, f in enumerate(testfiles):
        print(f)
        subset = None
        for i, chunk in enumerate(pd.read_csv(directory + f,
                                              usecols=cols[i],
                                              chunksize=50000,
                                              low_memory=False)):
            print(i)
            if subset is None:
                subset = chunk.copy()
            else:
                subset = pd.concat([subset, chunk])
            del chunk
            gc.collect()
        if testdata is None:
            testdata = subset.copy()
        else:
            testdata = pd.merge(testdata, subset.copy(), on="Id")
        del subset
        gc.collect()
        
    traindata = traindata.merge(df_mindate, on='Id')
    testdata = testdata.merge(df_mindate, on='Id')
        
    testdata['Response'] = 0  # Add Dummy Value
    visibletraindata = traindata[::2]
    blindtraindata = traindata[1::2]
    print(blindtraindata.columns)
    for i in range(2):
        for col in cols[i][1:]:
            print(col)
            blindtraindata.loc[:, col] = LeaveOneOut(visibletraindata,
                                                     blindtraindata,
                                                     col, False).values
            testdata.loc[:, col] = LeaveOneOut(visibletraindata,
                                               testdata, col, False).values
    del visibletraindata
    gc.collect()
    testdata.drop('Response', inplace=True, axis=1)
    return blindtraindata, testdata


def Train():
    train, test = GrabData()
    print('Train:', train.shape)
    print('Test', test.shape)
    features = list(train.columns)
    features.remove('Response')
    features.remove('Id')
    print(features)
    num_rounds = 50
    params = {}
    params['objective'] = "binary:logistic"
    params['eta'] = 0.021
    params['max_depth'] = 7
    params['colsample_bytree'] = 0.82
    params['min_child_weight'] = 3
    params['base_score'] = 0.005
    params['silent'] = True

    print('Fitting')
    trainpredictions = None
    testpredictions = None

    dvisibletrain = \
        xgb.DMatrix(train[features],
                    train.Response,
                    silent=True)
    dtest = \
        xgb.DMatrix(test[features],
                    silent=True)

    folds = 1
    for i in range(folds):
        print('Fold:', i)
        params['seed'] = i
        watchlist = [(dvisibletrain, 'train'), (dvisibletrain, 'val')]
        clf = xgb.train(params, dvisibletrain,
                        num_boost_round=num_rounds,
                        evals=watchlist,
                        early_stopping_rounds=20,
                        feval=mcc_eval,
                        maximize=True
                        )
        limit = clf.best_iteration+1
        # limit = clf.best_ntree_limit
        predictions = \
            clf.predict(dvisibletrain, ntree_limit=limit)

        best_proba, best_mcc, y_pred = eval_mcc(train.Response,
                                                predictions,
                                                True)
        print('tree limit:', limit)
        print('mcc:', best_mcc)
        print(matthews_corrcoef(train.Response,
                                y_pred))
        if(trainpredictions is None):
            trainpredictions = predictions
        else:
            trainpredictions += predictions
        predictions = clf.predict(dtest, ntree_limit=limit)
        if(testpredictions is None):
            testpredictions = predictions
        else:
            testpredictions += predictions
        imp = get_importance(clf, features)
        print('Importance array: ', imp)

    best_proba, best_mcc, y_pred = eval_mcc(train.Response,
                                            trainpredictions/folds,
                                            True)
    print(matthews_corrcoef(train.Response,
                            y_pred))

    submission = pd.DataFrame({"Id": train.Id,
                               "Prediction": trainpredictions/folds,
                               "Response": train.Response})
    submission[['Id',
                'Prediction',
                'Response']].to_csv('rawtrainxgbsubmission'+str(folds)+'.csv',
                                    index=False)

    submission = pd.DataFrame({"Id": test.Id.values,
                               "Response": testpredictions/folds})
    submission[['Id', 'Response']].to_csv('rawxgbsubmission'+str(folds)+'.csv',
                                          index=False)
    y_pred = (testpredictions/folds > .08).astype(int)
    submission = pd.DataFrame({"Id": test.Id.values,
                               "Response": y_pred})
    submission[['Id', 'Response']].to_csv('xgbsubmission'+str(folds)+'.csv',
                                          index=False)

if __name__ == "__main__":
    print('Started')
    Train()
    print('Finished')


ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyboardInterrupt: 